In [46]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import stats
import numpy.ma as ma
import cartopy.crs as ccrs
from tqdm import tqdm_notebook as tqdm

#import mplotutils as mpu
import cf_units
import cftime

In [2]:
##functions to calculate trends

def linregress_grid(df,df_time,start,end):
    x,y=np.meshgrid(np.arange(72),np.arange(144))
    slope=np.zeros([72,144])
    i_start=np.where(df_time==start)[0][0]
    i_end=np.where(df_time==end)[0][0]
    
    for i_x, i_y in zip(x.flatten(),y.flatten()):
        slope[i_x,i_y]=10*stats.linregress(np.arange(end-start+1),df[i_start:i_end+1,i_x,i_y])[0]
    return slope

def mean_trend_from_dict(df,df_time,start,end, roll=False):
    
    all_trends=np.stack([linregress_grid(df[key],df_time,start,end) for key in tqdm(df.keys())])
    
    if roll:
        return np.roll(np.mean(all_trends,axis=0),72,1)
    else:
        return np.mean(all_trends,axis=0)
    
def trend_dist_from_dict(df,df_time,start,end, roll=False):
    
    all_trends=np.stack([linregress_grid(df[key],df_time,start,end) for key in tqdm(df.keys())])
    
    if roll:
        return np.roll(np.quantile(all_trends,[0.05,0.95],axis=0),72,1)
    else:
        return np.quantile(all_trends,[0.05,0.95],axis=0)

In [40]:
##Getting nan mask from Knutson's dataset (stays the same for our time period)

df_1951_2010=xr.open_dataset('data/9cat.195101-201012.nc').roll(LON=72, roll_coords=True)
arr_1951_2010=np.full([72, 144],np.nan)

arr_1951_2010[(df_1951_2010.CP4.values==4).reshape(72, 144)]=3
arr_1951_2010[(df_1951_2010.CP3.values==3).reshape(72, 144)]=3
arr_1951_2010[(df_1951_2010.CP2.values==2).reshape(72, 144)]=2
arr_1951_2010[(df_1951_2010.CP1.values==1).reshape(72, 144)]=1
arr_1951_2010[(df_1951_2010.CP0.values==0).reshape(72, 144)]=0
arr_1951_2010[(df_1951_2010.CM4.values==-4).reshape(72, 144)]=-3
arr_1951_2010[(df_1951_2010.CM3.values==-3).reshape(72, 144)]=-3
arr_1951_2010[(df_1951_2010.CM2.values==-2).reshape(72, 144)]=-2
arr_1951_2010[(df_1951_2010.CM1.values==-1).reshape(72, 144)]=-1

nan_mask_knut=np.isnan(arr_1951_2010)



In [144]:
##Loading GPCC precip Observational data

df_gpcc=xr.open_mfdataset('data/precip.mon.total.2.5x2.5.v2018.nc').groupby('time.year').mean('time')
df_gpcc= df_gpcc.roll(lon=72).sortby('lat').assign_coords(lon= (((df_gpcc.lon + 180) % 360) - 180))
##lat is flipped for some reason

ref = 1951
trend_gpcc = {}
for year_idx in range(2000,2017):
        
    trend_gpcc[year_idx]=linregress_grid(df_gpcc.sel(year=slice(1870,2021)).precip.values,
                         df_gpcc.sel(year=slice(1870,2021)).year.values,ref,year_idx)
    trend_gpcc[year_idx][nan_mask_knut]=np.nan


In [145]:
# match [precipitation, years] to 2.5 degree grid, save as file
degrees = 2.5
lons = np.linspace(-180+degrees*.5,180-degrees*.5,int(360/degrees))
lats = np.linspace(-90+degrees*.5,90-degrees*.5,int(180/degrees))
xv, yv = np.meshgrid(lons,lats)


prec = list()
years = list()
lon = list()
lat = list()

for year in range(2000,2017):
    precs = trend_gpcc[year].flatten()
    prec.extend(precs)
    years.extend([year for i in range(len(precs))])
    lon.extend(list(xv.flatten()))
    lat.extend(list(yv.flatten()))
    
    
year_prec_grid = pd.DataFrame({"year": years, "precipitation": prec, "LON_25":lon,"LAT_25":lat})
year_prec_grid.to_csv("data/grid25_prec_years.csv")

In [146]:
# match fine subgrid with GDL code on coarse precipitation-trend data
def weighted_avg_with_nan(data, weights):
    if np.isnan(data).all():
        return np.nan
    masked = np.ma.masked_array(data, np.isnan(data))
    select = np.invert(np.ma.getmaskarray(masked))
    data = data[select]
    weights = weights[select]
    return np.average(data, weights=weights)

year_prec_grid = pd.read_csv("data/grid25_prec_years.csv")
grid_region = pd.read_csv('data/subgrid.csv')
#grid_region["LON_25"] = grid_region.LON_25.apply(lambda x: x+360 if x<0 else x)
grid_region = grid_region[['LAT_25','LON_25','GDLcode', 'area']]
year_prec_grid = year_prec_grid.merge(grid_region, on=["LAT_25", "LON_25"], how="left")
region_prec = year_prec_grid.groupby(["GDLcode","year"])\
              .apply(lambda x: weighted_avg_with_nan(data=x.precipitation, weights=x.area))\
              .reset_index().rename(columns={0:'precipitation_trend'})
region_prec.to_csv("data/region_prec_years.csv", index=False)

/tmp/ipykernel_54428/1685904747.py:12: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  grid_region = pd.read_csv('data/subgrid.csv')


In [149]:
# gif for grid
from cartopy import crs as ccrs
import cartopy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import imageio

def prec_trend_grid(year, grid_df):

    df = grid_df.loc[grid_df.year==year]
    fig = plt.figure(dpi=150, figsize=(7,4.5))

    ax = plt.subplot(projection=ccrs.EqualEarth())

    ax.coastlines(lw=0.1)

    n = np.array(df.precipitation).reshape(len(df.LAT_25.unique()),len(df.LON_25.unique()))

    mesh = ax.pcolormesh(
        df.LON_25.unique(),
        df.LAT_25.unique(),
        n,
        alpha=1,
        transform=ccrs.PlateCarree(),
        norm = mpl.colors.Normalize(vmin=-5,vmax=5),
        cmap="RdBu_r"
    )

    cbar = plt.colorbar(mesh, orientation="horizontal",
                       fraction=0.046, pad=0.04)
    cbar.set_label('Precipitation difference compared to 1951')

    ax.set_title(f'Precipitation Trend - Year {year}')
    fig.savefig(f"figures/grid25_prectrend1951_year{year}.png")
    plt.close()
    
    
grid_df = pd.read_csv("data/grid25_prec_years.csv")
for i in range(2000,2017):
    prec_trend_grid(i, grid_df)

filenames = [f"figures/grid25_prectrend1951_year{year}.png" for year in range(2000,2017)]
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('grid25_precipitation.gif', images, duration=0.5 )

In [150]:
# gif for region
import geoplot
import matplotlib.pyplot as plt
import matplotlib as mpl
import geopandas
from datetime import datetime

def plot_region_prec(year, dat):
    df = dat.loc[dat.year==year]
    
    fig, ax = plt.subplots(dpi=150, figsize=(7,4.5))
    ax = plt.subplot(projection=ccrs.EqualEarth())

    ax.coastlines(lw=0.1)

    cmap = mpl.cm.get_cmap('RdBu_r')
    norm = mpl.colors.Normalize(vmin=-5,vmax=5)

    #ax.add_geometries(merged.geometry, lw=0.1, linestyle=':',crs=ccrs.EqualEarth(),color=colors)
    for i, row in df.iterrows():

        ax.add_geometries(
            [row['geometry']],color=cmap(norm(row['precipitation_trend'])), 
            crs=ccrs.PlateCarree(),lw=0.1, linestyle=':',ec="black"
        )    
    cbar = fig.colorbar(
        mpl.cm.ScalarMappable(norm=norm, cmap=cmap), 
        ax=ax, orientation="horizontal",
        fraction=0.046, pad=0.04
    )
    ax.set_title(f'Precipitation Trend - Year {year}')
    cbar.set_label('Precipitation difference compared to 1951')
    fig.savefig(f"figures/region_prectrend1951_year{year}.png")
    plt.close()

gdl_shapes = geopandas.read_file("data/GDL Shapefiles V5 11-21.shp")
reg_df = pd.read_csv("data/region_prec_years.csv").merge(gdl_shapes,left_on="GDLcode",right_on="gdlcode").fillna(0)
for year in range(2000,2017):
    print(year)
    current_time = datetime.now().strftime("%H:%M:%S")
    print("Current Time =", current_time)
    plot_region_prec(year, reg_df)

2000
Current Time = 18:48:26
2001
Current Time = 19:08:53
2002
Current Time = 19:09:20
2003
Current Time = 19:09:47
2004
Current Time = 19:10:14
2005
Current Time = 19:10:40
2006
Current Time = 19:11:07
2007
Current Time = 19:11:34
2008
Current Time = 19:12:01
2009
Current Time = 19:12:28
2010
Current Time = 19:12:54
2011
Current Time = 19:13:21
2012
Current Time = 19:13:47
2013
Current Time = 19:14:14
2014
Current Time = 19:14:40
2015
Current Time = 19:15:07
2016
Current Time = 19:15:34


In [152]:
filenames = [f"figures/region_prectrend1951_year{year}.png" for year in range(2000,2017)]
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('region_precipitation.gif', images, duration=0.5 )

In [153]:
   
gen=6
usr_time_res='ann'
period=2016-1891
##create datasetsfor allforcing af, historical natural hn and preindustrial control piCONTROL
y_af={}
y_hn={}
y_pi={}


models=['MIROC6', 'IPSL-CM6A-LR', 'CanESM5', 'HadGEM3-GC31-LL', 'CNRM-CM6-1', 'GFDL-ESM4', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'NorESM2-LM', 'MRI-ESM2-0']

usable_models=[]
for model in models:
    
    
    dir_var_af='/net/atmos/data/cmip%i-ng/pr/'%gen
    if gen==6:
        member = 'r*i1p1f*'
        scenario = 'historical'
        hist_name = 'hist-nat'
        dir_var_af= dir_var_af + 'ann/g025/'
    elif gen==5:
        scenario = 'rcp85'
        hist_name = 'historicalNat'
        member = 'r*i1p1'
    
    #run_name_af_list=sorted(glob.glob(dir_var_af+'pr_%s_'%usr_time_res+model+'_'+scenario+'_'+member+'_g025.nc'))
    y_af[model]={}
    len_af=[]
    len_hn=[]
    len_pi=[]
    for i in range(6):
        
        if gen == 6:
            run_name_ssp=dir_var_af+'pr_%s_'%(usr_time_res)+model+'_ssp370_r1i1p1f1_g025.nc'
            print(model)
            print(run_name_ssp)
            if model=='HadGEM3-GC31-LL':
                run_name_ssp = dir_var_af+'pr_%s_'%(usr_time_res)+model+'_ssp245_r1i1p1f3_g025.nc'
                print(model)
                print(run_name_ssp)
            if model=='CNRM-CM6-1':
                run_name_ssp = dir_var_af+'pr_%s_'%(usr_time_res)  +model+'_ssp370_r1i1p1f2_g025.nc'
                print(model)
                print(run_name_ssp)
            if model=='GISS-E2-1-G':
                run_name_ssp = dir_var_af+'pr_%s_'%(usr_time_res) +model+'_ssp370_r6i1p1f2_g025.nc'
                print(model)
                print(run_name_ssp)

MIROC6
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_MIROC6_ssp370_r1i1p1f1_g025.nc
MIROC6
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_MIROC6_ssp370_r1i1p1f1_g025.nc
MIROC6
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_MIROC6_ssp370_r1i1p1f1_g025.nc
MIROC6
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_MIROC6_ssp370_r1i1p1f1_g025.nc
MIROC6
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_MIROC6_ssp370_r1i1p1f1_g025.nc
MIROC6
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_MIROC6_ssp370_r1i1p1f1_g025.nc
IPSL-CM6A-LR
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_IPSL-CM6A-LR_ssp370_r1i1p1f1_g025.nc
IPSL-CM6A-LR
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_IPSL-CM6A-LR_ssp370_r1i1p1f1_g025.nc
IPSL-CM6A-LR
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_IPSL-CM6A-LR_ssp370_r1i1p1f1_g025.nc
IPSL-CM6A-LR
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_IPSL-CM6A-LR_ssp370_r1i1p1f1_g025.nc
IPSL-CM6A-LR
/net/atmos/data/cmip6-ng/pr/ann/g025/pr_ann_IPSL-CM6A-LR_ssp370_r1i1p1f1_g025.nc
IPSL-CM6A-LR
/net/atmos/data/cmip6-ng/